<a href="https://colab.research.google.com/github/gabrielvcruz/projeto_reducao_de_dimensionalidade/blob/main/Sistema_de_recomenda%C3%A7%C3%A3o_por_imagens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install tensorflow matplotlib scikit-learn

# Importar bibliotecas
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.datasets import fashion_mnist

# Carregar o dataset Fashion MNIST
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalizar as imagens
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Exibir algumas imagens do dataset
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(x_train[i], cmap='gray')
    plt.axis('off')
plt.show()

# Carregar o modelo MobileNetV2 pré-treinado
base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')

# Função para extrair embedding
def extract_embedding(img_array):
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    embedding = base_model.predict(img_array)
    return embedding.flatten()

# Extrair embeddings para as imagens de treino e teste
train_embeddings = np.array([extract_embedding(x) for x in x_train])
test_embeddings = np.array([extract_embedding(x) for x in x_test])

print("Shape dos embeddings:", train_embeddings.shape)

# Função para recomendar imagens semelhantes
def recommend_similar_images(query_img, top_n=5):
    # Extrair o embedding da imagem de consulta
    query_embedding = extract_embedding(query_img)

    # Calcular a similaridade cosseno entre a imagem de consulta e todas as outras
    similarities = cosine_similarity([query_embedding], train_embeddings)

    # Obter as top_n imagens mais similares
    similar_indices = similarities.argsort()[0][-top_n:]

    # Exibir as imagens mais similares
    plt.figure(figsize=(10, 5))
    for i, idx in enumerate(similar_indices):
        plt.subplot(1, top_n, i + 1)
        plt.imshow(x_train[idx], cmap='gray')
        plt.axis('off')
    plt.show()

# Testar com uma imagem de consulta
recommend_similar_images(x_train[0], top_n=5)
